In [0]:
from google.colab import drive

drive.mount('/content/gdrive')

import numpy as np, math
with open('/content/gdrive/My Drive/datasets/ratings.train') as file:
    array2d = [[float(digit) for digit in line.split()] for line in file]
n=array2d[0]
n=int(n[0])
array2d=array2d[1:]
array2d=np.array(array2d)
train_data=array2d[:50000]
valid_set=array2d[50000:60000]
test_data=array2d[70001:n]
#print (valid_data)
train_data[:,2] *= .2
valid_set[:,2] *= .2
maxuser=0
for i in range(len(array2d)):
  if maxuser< array2d[i][0]:
    maxuser=array2d[i][0]
maxuser=int(maxuser)
print (maxuser)
maxitem=0
for i in range(len(array2d)):
  if maxitem< array2d[i][1]:
    maxitem=array2d[i][1]
maxitem=int(maxitem)
print (maxitem)
print(n)


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
609
9723
73562


In [0]:
import numpy as np
import math as m 
def valid_error(Z):
  err=0
  for i in range (len(valid_set)):
    err+=(Z[int(valid_set[i][0])][int(valid_set[i][1])]-valid_set[i][2])**2
    #print(err)
  print(m.sqrt(abs(err/len(valid_set))))

def soft_impute (train,lam,r,epsilon):
  Z=np.zeros((maxuser+1,maxitem+1))
  while 0 <1:
      Y=Z
      #print(len(train))
      for i in range (len(train)):
        Y[int(train[i][0])][int(train[i][1])]=train[i][2]
      #print(Y.shape)
      U,D,V=np.linalg.svd(Y)
      #print(D)
      D = [x - lam for x in D]
      #D-=lam
      #print(D)
      for i in range(r):
        D[i]=max(0,D[i])
      #Dcap=np.diag(D)
    
      Dcap=np.zeros((maxuser+1,maxitem+1))
      for i in range (r):
        Dcap[i][i]=D[i]
      #print(Dcap)
      Zold=Z
     # Z=np.matmul(U,Dcap)
      #print(Z.shape)
      #Z=np.matmul(Z,np.transpose(V))
      #print(Z.shape)
      Z=U @ Dcap @ V
      old_norm=np.linalg.norm(Zold,ord='fro')
      Z_norm=np.linalg.norm(Z-Zold,ord='fro')
      #print(Z_norm)
      print(Z)
      train_data[:,2] *= 5
      train_data[:,2] *= 5
      valid_error(Z)
      # if Z_norm < old_norm*epsilon:
      #   break
      
      #return Z
  return Z

In [0]:
Z=soft_impute(train_data,0.00001,16,1)



[[ 7.62098986e+00  1.25730401e+00  2.25483614e+00 ... -3.40831982e-01
   0.00000000e+00 -3.97637313e-01]
 [-3.31443741e-02 -1.33628119e-02  5.44031921e-01 ... -7.20310600e-02
   0.00000000e+00 -8.40362367e-02]
 [ 9.23467196e-02 -6.14905430e-03  5.20451681e-02 ...  9.94433655e-02
   0.00000000e+00  1.16017260e-01]
 ...
 [ 6.92627774e+00  3.69366352e+00  6.12477556e+00 ... -1.68432266e-01
   0.00000000e+00 -1.96504311e-01]
 [ 2.64636105e+00  5.44562254e-01  1.64866932e+00 ...  1.00465164e-01
   0.00000000e+00  1.17209358e-01]
 [ 1.89604494e+00 -4.40785094e-01 -7.57497869e-01 ...  1.42250145e+01
   0.00000000e+00  1.65958502e+01]]
2.8496235135979027
[[ 1.90834616e+02  3.14975922e+01  5.67605352e+01 ... -8.53277847e+00
   0.00000000e+00 -9.95490822e+00]
 [-7.89511446e-01 -2.93279848e-01  1.37380903e+01 ... -1.78941711e+00
   0.00000000e+00 -2.08765330e+00]
 [ 2.26572140e+00 -1.61775239e-01  1.29473953e+00 ...  2.48463595e+00
   0.00000000e+00  2.89874195e+00]
 ...
 [ 1.70908285e+02  9.2615

KeyboardInterrupt: ignored

In [0]:
'''
   Copyright 2015 Travis Brady
   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at
       http://www.apache.org/licenses/LICENSE-2.0
   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
'''
from __future__ import print_function
import numpy as np


def frob(Uold, Dsqold, Vold, U, Dsq, V):
    denom = (Dsqold ** 2).sum()
    utu = Dsq * (U.T.dot(Uold))
    vtv = Dsqold * (Vold.T.dot(V))
    uvprod = utu.dot(vtv).diagonal().sum()
    num = denom + (Dsqold ** 2).sum() - 2*uvprod
    return num / max(denom, 1e-9)

class SoftImpute:
    def __init__(self, J=2, thresh=1e-05, lambda_=0, maxit=100, random_state=None, verbose=False):
        self.J = J
        self.thresh = thresh
        self.lambda_ = lambda_
        self.maxit = maxit
        self.rs = np.random.RandomState(random_state)
        self.verbose = verbose
        self.u = None
        self.d = None
        self.v = None

    def fit(self, X):
        n,m = X.shape
        xnas = np.isnan(X)
        nz = m*n - xnas.sum()
        xfill = X.copy()
        V = np.zeros((m, self.J))
        U = self.rs.normal(0.0, 1.0, (n, self.J))
        U, _, _ = np.linalg.svd(U, full_matrices=False)
        Dsq = np.ones((self.J, 1))
        #xfill[xnas] = 0.0
        col_means = np.nanmean(xfill, axis=0)
        np.copyto(xfill, col_means, where=np.isnan(xfill))
        ratio = 1.0
        iters = 0
        while ratio > self.thresh and iters < self.maxit:
            iters += 1
            U_old = U
            V_old = V
            Dsq_old = Dsq
            B = U.T.dot(xfill)

            if self.lambda_ > 0:
                tmp = (Dsq / (Dsq + self.lambda_))
                B = B * tmp

            Bsvd = np.linalg.svd(B.T, full_matrices=False)
            V = Bsvd[0]
            Dsq = Bsvd[1][:, np.newaxis]
            U = U.dot(Bsvd[2])

            tmp = Dsq * V.T

            xhat = U.dot(tmp)

            xfill[xnas] = xhat[xnas]
            A = xfill.dot(V).T
            Asvd = np.linalg.svd(A.T, full_matrices=False)
            U = Asvd[0]
            Dsq = Asvd[1][:, np.newaxis]
            V = V.dot(Asvd[2])
            tmp = Dsq * V.T

            xhat = U.dot(tmp)
            xfill[xnas] = xhat[xnas]
            ratio = frob(U_old, Dsq_old, V_old, U, Dsq, V)
            if self.verbose:
                print('iter: %4d ratio = %.5f' % (iters, ratio))

        self.u = U[:,:self.J]
        self.d = Dsq[:self.J]
        self.v = V[:,:self.J]
        return self

    def suv(self, vd):
        res = self.u.dot(vd.T)
        return res

    def predict(self, X, copyto=False):
        vd = self.v * np.outer(np.ones(self.v.shape[0]), self.d)
        X_imp = self.suv(vd)
        if copyto:
            np.copyto(X, X_imp, where=np.isnan(X))
        else:
            return X_imp

def main():
    X = np.random.random((10,3)) + (np.arange(10).reshape(10,1) ** 2)

    clf = SoftImpute(J=2, lambda_=0.0)
    fit = clf.fit(X)
    X_test = X.copy()
    X_test[3,1] = np.nan
    X_imp = clf.predict(X_test)

if __name__ == '__main__':
    main()



print(MATRIX)
SoftImpute(J=2, lambda_=0.0).fit(MATRIX)
imputed = SoftImpute(J=2, lambda_=0.0).predict(MATRIX)
v

SyntaxError: ignored

In [0]:
def rmse(Z):
    s = 0
    for i in range(len(valid_set)):
        user = int(valid_set[i][0])
        item = int(valid_set[i][1])
        rating = valid_set[i][2]
       
        error = rating - Z[user, item]
        error = error*error
        s = s + error
    a = s/len(valid_set)
    a = math.sqrt(a)
    return a

In [0]:
from fancyimpute import SoftImpute,BiScaler
MATRIX=np.empty((maxuser+1,maxitem+1))
MATRIX[:] = np.nan
for i in train_data:
  MATRIX[int(i[0])][int(i[1])]=i[2]
MATRIX=np.array(MATRIX)
#X_incomplete_normalized = BiScaler().fit_transform(MATRIX)
X_filled_softimpute = SoftImpute().fit_transform(MATRIX)
valid_error(imputed)

[SoftImpute] Max Singular Value of X_init = 261.177736
[SoftImpute] Iter 1: observed MAE=0.423502 rank=589
[SoftImpute] Iter 2: observed MAE=0.420651 rank=555
[SoftImpute] Iter 3: observed MAE=0.416668 rank=525
[SoftImpute] Iter 4: observed MAE=0.412486 rank=497
[SoftImpute] Iter 5: observed MAE=0.408467 rank=474
[SoftImpute] Iter 6: observed MAE=0.404608 rank=452
[SoftImpute] Iter 7: observed MAE=0.401053 rank=434
[SoftImpute] Iter 8: observed MAE=0.397729 rank=417
[SoftImpute] Iter 9: observed MAE=0.394688 rank=402
[SoftImpute] Iter 10: observed MAE=0.391819 rank=391
[SoftImpute] Iter 11: observed MAE=0.389265 rank=378
[SoftImpute] Iter 12: observed MAE=0.386811 rank=367
[SoftImpute] Iter 13: observed MAE=0.384451 rank=356
[SoftImpute] Iter 14: observed MAE=0.382244 rank=348
[SoftImpute] Iter 15: observed MAE=0.380175 rank=338
[SoftImpute] Iter 16: observed MAE=0.378236 rank=332
[SoftImpute] Iter 17: observed MAE=0.376359 rank=323
[SoftImpute] Iter 18: observed MAE=0.374593 rank=316


NameError: ignored

In [0]:
print(rmse(X_filled_softimpute))

2.154230899036549
